# Libs

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics import r2_score
import plotly.graph_objects as go
import plotly.io as pio
from tqdm import tqdm

pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)

# Data uploading

In [3]:
# def metadata_parquet_loading():
#     path = 'C:\\jupyter\\SPP\\input\\'
#     metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
#     metadata = metadata_init.copy()
#     metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
#     metadata.Status = metadata.Status.str.strip()
#     metadata.Status = metadata.Status.str.lower()
#     metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
#     metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
#     metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
#     metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
#     metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
#     metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
#     metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
#     metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
#     metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
#     metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
#     metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
#     metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
#     metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
#     metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
#     metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
#     metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
#     metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
#     metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
#     metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
#     metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
#     metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
#     metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
#     metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'

#     #Read data from parquet
#     path = 'C:\\jupyter\\SPP\\input\\'
#     df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v10.parquet.gzip')
#     df_prq.rename(columns={'wellName':'well'}, inplace=True)
#     df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
#     # Filter data with bad_well_list 
#     bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP', 'A01W_2']
#     df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
#     #Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
#     df_prq.loc[df_prq.FLUIDS>0, 'FLUIDS_int'] = 1
#     df_prq.loc[df_prq.FLUIDS<=0, 'FLUIDS_int'] = 0
#     df_prq.FLUIDS_int = df_prq.FLUIDS_int.astype('int')
#     # Unite of FU for each formation
#     df_bal = df_prq[df_prq.FORMATION.str.contains('Balakhany')]
#     df_bal.loc[df_bal.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
#     df_bal.loc[df_bal.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
#     df_bal = df_bal[df_bal.FORMATION_up.notna()]
#     #Getting XY mean coords of Balakhany formation
#     xy_coord_mean = df_bal[['well', 'FORMATION_up', 'X', 'Y']]
#     xy_coord_mean = xy_coord_mean.groupby(['well', 'FORMATION_up']).agg({'X': 'mean', 'Y':'mean'}).reset_index()
#     xy_coord_mean = xy_coord_mean.rename(columns={'X':'X_mean', 'Y':'Y_mean'})
#     xy_coord_mean = xy_coord_mean[xy_coord_mean.FORMATION_up.str.contains('Balakhany') & (xy_coord_mean.X_mean>0) & (xy_coord_mean.Y_mean>0)]
#     df_bal.rename(columns={'X':'X_traj', 'Y':'Y_traj'}, inplace=True)
#     df_bal = df_bal.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()
#     return df_bal
# df_bal = metadata_parquet_loading()

In [6]:
core = pd.read_csv(r'C:\jupyter\SPP_uncertainty\raw\CoreDB_bal.csv')
core['Core Depth'] = core['Core Depth'].round(1)
logs8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\logs8_ntd_v5.csv')
logs10 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\logs10_ntd_v4.csv')
logs = pd.concat([logs8, logs10], axis=0)

In [ ]:
core.columns

In [ ]:
core.Well.unique()

# Gas wells

## Bal8

In [25]:
df_bal8_v4_flag = logs8

In [ ]:
gas_wells = df_bal8_v4_flag.groupby(['well', 'fluid_code'])['tst'].count().reset_index().rename(columns={'tst':'fluid_code_points'})
total = df_bal8_v4_flag.groupby('well')['tst'].count().reset_index().rename(columns={'tst':'total_points'})
gas_wells_v2 = gas_wells.set_index('well').join(total.set_index('well')).reset_index()
gas_wells_v2['fluid_code%'] = gas_wells_v2['fluid_code_points']/gas_wells_v2['total_points']
gas_wells_v2 = gas_wells_v2[(gas_wells_v2.fluid_code == 1) & (gas_wells_v2['fluid_code%'] > 0.5)]
names_gas_wells8_v2 = gas_wells_v2.well.unique()
names_gas_wells8_v2

In [ ]:
for wellname in core.Well.unique():
    if wellname in names_gas_wells8_v2:
        print('gas well:', wellname)
    else:
        print('no gas bal8:', wellname)

## Bal10

In [16]:
df_bal10_v4_flag = logs10

In [ ]:
gas_wells = df_bal10_v4_flag.groupby(['well', 'fluid_code'])['tst'].count().reset_index().rename(columns={'tst':'fluid_code_points'})
total = df_bal10_v4_flag.groupby('well')['tst'].count().reset_index().rename(columns={'tst':'total_points'})
gas_wells_v2 = gas_wells.set_index('well').join(total.set_index('well')).reset_index()
gas_wells_v2['fluid_code%'] = gas_wells_v2['fluid_code_points']/gas_wells_v2['total_points']
gas_wells_v2 = gas_wells_v2[(gas_wells_v2.fluid_code == 1) & (gas_wells_v2['fluid_code%'] > 0.5)]
names_gas_wells10_v2 = gas_wells_v2.well.unique()
names_gas_wells10_v2

In [ ]:
core.Well.unique()

In [ ]:
for wellname in core.Well.unique():
    if wellname in names_gas_wells10_v2:
        print('gas well:', wellname)
    else:
        print('no gas bal8:', wellname)

# K-phit

In [ ]:
core_phit_k = core[core.Well.isin(['J02', 'D05ST1','D01Z', 'C01','B02Z', 'B01ST1', 'A22'])]
core_phit_k = core_phit_k[[ 'Well', 'Formation', 'Core Depth', 
                            'CPOR_OBC', 'CPERM_OBC', 
                            # 'Facies Class', 'Facies Undifferentiated', 'Facies Type',
                            'CPOR_H', 'CPOR_V','CKHG', 'CKVG', 
                            'GRAIN_DEN_H', 'GRAIN_DEN_V']]
# core_phit_k.to_csv('core_phit_k.csv', index=False)
core_obc = core_phit_k.copy()
core_obc = core_obc[core_obc.CPERM_OBC >= 0.1]
# core_phit_k = core_phit_k[core_phit_k.CPOR_H < 0.4]
fig, ax = plt.subplots(1,3, figsize=(18, 6))
sns.scatterplot(data=core_obc, x='CPOR_OBC', y='CPERM_OBC', hue='Formation', ax=ax[0])
ax[0].set_yscale('log')
ax[0].grid(True)
sns.histplot(data=core_phit_k, x='GRAIN_DEN_H', hue='Formation', ax=ax[1], kde=True, bins=15)
ax[1].grid(True)
sns.histplot(data=core_phit_k, x='GRAIN_DEN_V', hue='Formation', ax=ax[2], kde=True, bins=20)
ax[2].grid(True)

In [ ]:
def core_phit_all(core_phit_k):
    por_v = core_phit_k[core_phit_k.CPOR_V.notna()][['Well', 'Formation', 'Core Depth','CPOR_V','CKVG']].rename(columns={'CPOR_V':'CPOR', 'CKVG':'CPERM'})
    por_h = core_phit_k[core_phit_k.CPOR_H.notna()][['Well', 'Formation', 'Core Depth','CPOR_H','CKHG']].rename(columns={'CPOR_H':'CPOR', 'CKHG':'CPERM'})
    core_obc_v2 = core_phit_k[core_phit_k.CPERM_OBC.notna()][['Well', 'Formation', 'Core Depth','CPOR_OBC', 'CPERM_OBC']].rename(columns={'CPOR_OBC':'CPOR', 'CPERM_OBC':'CPERM'})
    por_h['type'] = 'H'
    por_v['type'] = 'V'
    core_obc_v2['type'] = 'OBC'
    full_core_table = pd.concat([por_h, por_v, core_obc_v2], axis=0)
    full_core_table = full_core_table[(full_core_table.CPERM < 1000) & (full_core_table.CPOR <0.35)]
    fig, ax = plt.subplots(figsize=(9, 7))
    # sns.scatterplot(data=full_core_table[full_core_table.type != 'OBC'], x='CPOR', y='CPERM', hue='type', style='Formation', s=50, alpha=0.7)
    sns.scatterplot(data=full_core_table, x='CPOR', y='CPERM', hue='type', style='Formation', s=50, alpha=0.7)
    plt.yscale('log')
    plt.grid(True);
core_phit_all(core_phit_k)

# MICP

In [9]:
# PSD y vs Radius x
# PTR vs PSD


In [ ]:
core_hg = core[['Well', 'Formation', 'Core Depth', 'Input Pressure', 'HG Saturation', 'Normalized PTR Distribution (dS/dlogR)', 'PSD', 'Radius',
                'CPOR_H', 'CPOR_V', 'CKHG', 'CKVG']]
wellname = 'G01Z'
fig, ax = plt.subplots(1,2, figsize=(15, 6))
sns.scatterplot(data=core_hg[core_hg.Well == wellname], 
                x='HG Saturation', y='Input Pressure', hue='Core Depth', style='Formation', palette='viridis_r', alpha=0.75, ax=ax[0])
ax[0].grid(True)
ax[0].set_xlim(1.05, 0.5)
sns.lineplot(data=core_hg[(core_hg.Well == wellname) 
                          & (core_hg.Radius.notna())], x='Radius', y='PSD', hue='Core Depth', style='Formation', palette='viridis_r', ax=ax[1])
ax[1].grid(True)
well_hg_depth = core_hg[core_hg['HG Saturation'].notna()].groupby(['Well','Core Depth'])['HG Saturation'].count().reset_index()
well_hg_depth.groupby('Well')['Core Depth'].count().reset_index().set_index('Well').T

In [ ]:
test = well_hg_depth.sort_values(['Well','Core Depth'])
test[test.Well == 'C01']

# Core well matching

In [ ]:
core_phit_k.Well.unique()

In [ ]:
def well_display_logs_core(dataset, core_data, wellname, fmname, height_coeff, var1, var2, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    data_net_khtst = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    data_core = core_data[(core_data.Well == wellname) & (core_data.Formation == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data[var1]*150
    net_init_calc = data_net_khtst[var1]
    net = data[var2]*100
    net_calc = data_net_khtst[var2]
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    khtst = data['KHtst']

    cdepth = data_core['Core Depth']
    cphit = data_core['CPOR_H'] # CPOR_H CPOR_OBC
    cperm = data_core['CKHG'] # CKHG CPERM_OBC

    var1_sum = (net_init_calc.sum()*0.1).round(0)
    var2_sum = (net_calc.sum()*0.1).round(0)
    tst_interval = round(depth.max() - depth.min(),0)
    height = int(tst_interval/height_coeff)
    khtst_top = int(data_net_khtst['KHtst'].iloc[0])

    fig, ax = plt.subplots(1,3, figsize=(6,height), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[0].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    # ax[0].set_title((var1 + ':' + str(var1_sum) + 'm'), fontsize=10)
    ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin0 = ax[0].twiny()
    # twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    # twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    # twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    # twin0.set_xlim(0, 1.5)
    # twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.15)
    twin0.set_xlim(0, 1)
    # ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    # ax[1].invert_yaxis() 
    # ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    # ax[1].grid(True)
    # ax[1].set_xlim(1.65, 2.65)
    # for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
    #     ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
    #     xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
    #     ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    # twin1 = ax[1].twiny()
    # twin1.plot(npss, depth, color='blue', alpha=0.75)
    # twin1.set_xlim(0.6, 0)
    # twin1.grid(False)
    ax[1].plot(phit, depth, color='green', linestyle='dashed', zorder=5)
    ax[1].scatter(cphit, cdepth, color='blue', marker='o', alpha=0.25, zorder=2)
    ax[1].set_xlim(0.3, 0)
    ax[1].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[1].tick_params(labelsize=10)
    ax[1].grid(True)
    ax[1].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[1].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    # ax[2].set_title((var2 + ':' + str(var2_sum) + 'm'), fontsize=10)
    ax[1].set_title(var2 + ':' + str(var2_sum), fontsize=10)
    # ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin2 = ax[1].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1, zorder=2)
    twin2.fill_betweenx(depth, net, color='orange', alpha=0.15, zorder=2)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    # twin3 = ax[2].twiny()
    # twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    # twin3.set_xscale('log')
    # twin3.set_xlim(1, 200)
    # twin3.grid(False)
    ax[2].plot(perm, depth, color='purple', alpha=0.75)
    ax[2].scatter(cperm, cdepth, color='red')
    ax[2].grid(True)
    # ax[2].plot(perm1, depth, color='purple', alpha=0.5)
    # ax[2].plot(perm2, depth, color='purple', alpha=0.5)
    twin4 = ax[2].twiny()
    twin4.plot(khtst, depth, color='black', alpha=1)
    ax[2].set_xscale('log')
    ax[2].set_xlim(0.1, 1000)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' 
                + ref_depth +':' + str(tst_interval) + 
                ' KHtst:' + str(khtst_top), fontsize=12)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show();

c01_8 = core_phit_k[(core_phit_k.Well == 'C01') & (core_phit_k.Formation == 'Balakhany VIII')]
c01_8['Core Depth'] = c01_8['Core Depth'] + 3
well_display_logs_core(logs, c01_8, 'C01', 'Balakhany VIII', 15, 'NET_VSH', 'NET', 'MD', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
b02z = core_phit_k[core_phit_k.Well == 'B02Z']
b02z['Core Depth'] = b02z['Core Depth'] + 25
well_display_logs_core(logs, b02z, 'B02Z', 'Balakhany VIII', 20, 'NET_VSH', 'NET', 'MD', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
d05st1 = core_phit_k[core_phit_k.Well == 'D05ST1']
d05st1['Core Depth'] = d05st1['Core Depth'] + 20
well_display_logs_core(logs, d05st1, 'D05ST1', 'Balakhany VIII', 20, 'NET_VSH', 'NET', 'MD', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')

well_display_logs_core(logs, core_phit_k, 'A22', 'Balakhany X', 12, 'NET_VSH', 'NET', 'MD', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
b01st1 = core_phit_k[core_phit_k.Well == 'B01ST1']
b01st1['Core Depth'] = b01st1['Core Depth'] + 24
well_display_logs_core(logs, b01st1, 'B01ST1', 'Balakhany X', 12, 'NET_VSH', 'NET', 'MD', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
c01_10 = core_phit_k[(core_phit_k.Well == 'C01') & (core_phit_k.Formation == 'Balakhany X')]
c01_10['Core Depth'] = c01_10['Core Depth']+3
well_display_logs_core(logs, c01_10, 'C01', 'Balakhany X', 20, 'NET_VSH', 'NET', 'MD', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')
j02 = core_phit_k[core_phit_k.Well == 'J02']
j02['Core Depth'] = j02['Core Depth'] + 23
well_display_logs_core(logs, j02, 'J02', 'Balakhany X', 8, 'NET_VSH', 'NET', 'MD', 10, 'dontprint', 'C:\\jupyter\\SPP\\output\\')

In [14]:
core_kphi_tied = pd.concat([c01_8, b02z, d05st1, b01st1, c01_10, j02], axis=0)
core_kphi_tied_v2 = core_kphi_tied.drop_duplicates(subset=['Core Depth']).sort_values(
                        by=['Well', 'Formation', 'Core Depth']).reset_index(drop=True)

In [15]:
core_values = core_kphi_tied_v2[['CPOR_OBC', 'CPERM_OBC', 'CPOR_H', 'CPOR_V', 'CKHG', 'CKVG', 'GRAIN_DEN_H', 'GRAIN_DEN_V']]
core_values = core_values.dropna(axis=0, how='all')
core_values = core_values.fillna(0)
core_values['CORE_RES'] = 'res'
wells_depth = core_kphi_tied_v2[['Well', 'Formation', 'Core Depth']]
wells_depth2 = wells_depth.join(core_values, how='inner').reset_index(drop=True)
wells_depth2 = wells_depth2.rename(columns={'Core Depth':'MD', 'Formation':'FORMATION_up', 'Well':'well'}).set_index(['well','FORMATION_up','MD'])
logs_core = logs.set_index(['well','FORMATION_up','MD']).join(wells_depth2).reset_index()
logs_core.loc[(logs_core.NET == 0) & (logs_core.CPOR_H.notna()), 'CORE_RES'] = 'non-res'
logs_core= logs_core.fillna(0)
logs_core.loc[(logs_core.CORE_RES == 0), 'CORE_RES'] = 'empty'
logs_core['CORE_RES'] = logs_core['CORE_RES'].astype('string')

In [16]:
logs_core = logs_core.astype({'well': 'string', 
                              'FORMATION_up': 'string', 
                              'FORMATION': 'string',
                              'field': 'string'})
# logs_core.drop('V', axis=1).to_csv('logs_core.csv', index=False)

In [ ]:
core_tvd = logs_core[['well','MD','FORMATION_up','PHIT','CPOR_OBC','CPOR_H', 'CPOR_V', 'TVD_SCS', 'NET', 'VSH']]
core_cporh = core_tvd[core_tvd.CPOR_H > 0]
core_cporv = core_tvd[core_tvd.CPOR_V > 0]
core_obc = core_tvd[core_tvd.CPOR_OBC > 0]

depthh = core_cporh['TVD_SCS']
por_h = core_cporh['CPOR_H']
vsh_h = core_cporh['VSH']
depthv = core_cporv['TVD_SCS']
por_v = core_cporv['CPOR_V']
vsh_v = core_cporv['VSH']
depth_obc = core_obc['TVD_SCS']
por_obc = core_obc['CPOR_OBC']
vsh_obc = core_obc['VSH']

def regress_line(x, y):
    coefficients = np.polyfit(x, y, 1)
    polynomial = np.poly1d(coefficients)
    y_fit = polynomial(x)
    print(f'{y.name} = {coefficients[0]:.6f} * {x.name} + {coefficients[1]:.3f}')
    return y_fit
cporh_fit = regress_line(depthh, por_h)
cporv_fit = regress_line(depthv, por_v)
cporobc_fit = regress_line(depth_obc, por_obc)

fig, ax = plt.subplots(1,3, figsize=(10, 9))
sns.kdeplot(data=core_cporh, x='CPOR_H', y='TVD_SCS', color='blue', alpha=0.55, ax=ax[0])
ax[0].scatter(y=depthh, x=por_h, c=vsh_h.values, s=vsh_h*75, alpha=0.55, ec='black', cmap='viridis_r')
sns.kdeplot(data=core_cporv, x='CPOR_V', y='TVD_SCS', color='blue', alpha=0.55, ax=ax[1])
ax[1].scatter(y=depthv, x=por_v, c=vsh_v.values, s=vsh_v*75, alpha=0.55, ec='black', cmap='viridis_r')
sns.kdeplot(data=core_obc, x='CPOR_OBC', y='TVD_SCS', color='blue', alpha=0.55, ax=ax[2])
test = ax[2].scatter(y=depth_obc, x=por_obc, c=vsh_obc.values, s=vsh_obc*75, alpha=0.55, ec='black', cmap='viridis_r')
plt.colorbar(test, label='VSH')
sns.lineplot(x=cporh_fit, y=depthh, color='red', lw=2, ax=ax[0])
sns.lineplot(x=cporv_fit, y=depthv, color='red', lw=2, ax=ax[1])
sns.lineplot(x=cporobc_fit, y=depth_obc, color='red', lw=2, ax=ax[2])

ax[0].invert_yaxis()
ax[0].grid(True)
ax[0].set_title('CPOR_H ' + str((depthh.max() - depthh.min()).round(0)) + 'm')
ax[1].invert_yaxis()
ax[1].grid(True)
ax[1].set_title('CPOR_V ' + str((depthv.max() - depthv.min()).round(0)) + 'm')
ax[2].invert_yaxis()
ax[2].grid(True)
ax[2].set_title('CPOR_OBC ' + str((depth_obc.max() - depth_obc.min()).round(0)) + 'm' );

# Xplots core/logs

In [ ]:
def phit_logs_core(logs_core):
    def regress_line(logs_core, var):
        data = logs_core[(logs_core[var] !=0) & (logs_core.NET==1)]
        x = data[var]
        y = data.PHIT
        coefficients = np.polyfit(x, y, 1)
        polynomial = np.poly1d(coefficients)
        y_fit = polynomial(data[var])
        r2 = r2_score(y, y_fit).round(2)
        return x, y_fit, coefficients, r2
    x_obc, y_fit_obc, coef_obc, r2_obc = regress_line(logs_core, 'CPOR_OBC')
    x_h, y_fit_h, coef_h, r2_h = regress_line(logs_core, 'CPOR_H')

    fig, ax = plt.subplots(2,2, figsize=(15, 12))
    sns.scatterplot(data=logs_core[(logs_core.CPOR_OBC !=0) & (logs_core.NET==1)], x='CPOR_OBC', y='PHIT', hue='well', style='FORMATION_up', 
                    s=50, ax=ax[0,0])
    sns.lineplot(x=[0, 0.4], y=[0, 0.4], color='black', linestyle='dashed', ax=ax[0,0])
    ax[0,0].set_title('NET=1'), ax[0,0].grid(True)
    sns.lineplot(x=x_obc, y=y_fit_obc, color='blue', ax=ax[0,0])
    ax[0,0].text(0.15, 0.1, f'y = {coef_obc[0]:.2f}x + {coef_obc[1]:.2f}  r2:{r2_obc}')

    sns.scatterplot(data=logs_core[(logs_core.CPOR_OBC !=0) & (logs_core.NET==0)], x='CPOR_OBC', y='PHIT', hue='well', style='FORMATION_up', s=50, ax=ax[0,1])
    sns.lineplot(x=[0, 0.4], y=[0, 0.4], color='black', linestyle='dashed', ax=ax[0,1])
    ax[0,1].set_title('NET=0'),  ax[0,1].grid(True)

    sns.scatterplot(data=logs_core[(logs_core.CPOR_H !=0) & (logs_core.NET==1)], x='CPOR_H', y='PHIT', hue='well', style='FORMATION_up', s=50, ax=ax[1,0])
    sns.lineplot(x=[0, 0.4], y=[0, 0.4], color='black', linestyle='dashed', ax=ax[1,0])
    sns.lineplot(x=[0.025, 0.425], y=[0, 0.4], color='black', linestyle='dotted', ax=ax[1,0])
    ax[1,0].set_title('NET=1'),  ax[1,0].grid(True)
    sns.lineplot(x=x_h, y=y_fit_h, color='blue', ax=ax[1,0])
    ax[1,0].text(0.15, 0.1, f'y = {coef_h[0]:.2f}x + {coef_h[1]:.2f} r2:{r2_h}')

    sns.scatterplot(data=logs_core[(logs_core.CPOR_V !=0) & (logs_core.NET==0)], x='CPOR_V', y='PHIT', hue='well', style='FORMATION_up', s=50, ax=ax[1,1])
    sns.lineplot(x=[0, 0.4], y=[0, 0.4], color='black', linestyle='dashed', ax=ax[1,1])
    sns.lineplot(x=[0.035, 0.435], y=[0, 0.4], color='black', linestyle='dotted', ax=ax[1,1])
    ax[1,1].set_title('NET=0'), ax[1,1].grid(True);
phit_logs_core(logs_core)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(18, 9))

logs_core_zero = logs_core[(logs_core.CORE_RES != 'empty')]
sns.scatterplot(data=logs_core_zero, x='CPOR_H', y='CKHG', hue='well', style='FORMATION_up', s=100, ax=ax[0])
ax[0].set_yscale('log')
ax[0].set_xlim(0, 0.35)
ax[0].set_ylim(0.01, 7000)
ax[0].set_title('non-res kphit')

logs_core_net = logs_core[(logs_core.CORE_RES != 'empty')]
sns.scatterplot(data=logs_core_net, x='CPOR_H', y='CKHG', hue='CORE_RES', style='FORMATION_up', ax=ax[1], alpha=0.75)
ax[1].set_yscale('log')
ax[1].set_xlim(0, 0.35)
ax[1].set_ylim(0.01, 7000)
ax[1].set_title('res vs non-res kphit');

# Vsh_cutoff_uncert

In [ ]:
def net_vsh(dataset, cutoff):
    var = 'NET_VSH' + '_' + str(cutoff).split('.')[1]
    dataset[var] = 0
    dataset.loc[dataset.VSH <= cutoff, var] = 1
    new_net_vsh = dataset.groupby(['well','FORMATION_up'])[var].sum().reset_index()
    net_vsh = dataset.groupby(['well','FORMATION_up'])['NET_VSH'].sum().reset_index()
    net_vsh['NET_VSH'] = net_vsh['NET_VSH']*0.1
    new_net_vsh[var] = new_net_vsh[var]*0.1
    result = net_vsh.merge(new_net_vsh, on=['well','FORMATION_up'], how='inner')
    return result, var, dataset, cutoff
bal8, var8, logs8, cutoff8 = net_vsh(logs[logs.FORMATION_up == 'Balakhany VIII'], 0.61)
bal10, var10, logs10, cutoff10 = net_vsh(logs[logs.FORMATION_up == 'Balakhany X'], 0.56)

def xplot_net_vsh(bal8, var8, bal10, var10):
    fig, ax = plt.subplots(1,2, figsize=(10, 4))
    sns.scatterplot(data=bal8, x='NET_VSH', y=var8, s=30, ax=ax[0], c='orange', alpha=0.75, ec='black')
    sns.scatterplot(data=bal10, x='NET_VSH', y=var10, s=30, ax=ax[1], c='green', alpha=0.75, ec='black')
    sns.lineplot(x=[0, 100], y=[0, 100], color='black', linestyle='dashed', ax=ax[0])
    sns.lineplot(x=[0, 100], y=[0, 114], color='red', linestyle='dashed', ax=ax[0])
    sns.lineplot(x=[0, 100], y=[0, 100], color='black', linestyle='dashed', ax=ax[1])
    sns.lineplot(x=[0, 100], y=[0, 114], color='red', linestyle='dashed', ax=ax[1])
    ax[0].text(65, 65, 'line 1=1')
    ax[0].text(60, 90, 'y=1.14x', fontsize=10, c='red')
    ax[0].grid(True)
    ax[1].text(65, 65, 'line 1=1')
    ax[1].text(60, 90, 'y=1.14x', fontsize=10, c='red')
    ax[1].grid(True)
xplot_net_vsh(bal8, var8, bal10, var10)

def vsh_net_well_display(logs8, var8, cutoff8):
    my_list = list(logs8.well.unique())
    selected_samples = random.sample(my_list, 4)
    fig, ax = plt.subplots(1,4, figsize=(10, 5))
    def vsh_net_track(wellname, dataset, var, cutoff, num):
        wells = dataset[(dataset.well == wellname)].iloc[300:500]
        vsh = wells['VSH']
        tst = wells['TST']
        net_init = wells['NET_VSH']*0.45
        net = wells[var]*cutoff
        ax[num].plot(vsh, tst, color='green', zorder=3)
        ax[num].plot(net_init, tst, color='black', lw=0.5)
        ax[num].plot(net, tst, color='black', lw=0.5)
        ax[num].fill_betweenx(tst,net_init, color='orange', alpha=0.5)
        ax[num].fill_betweenx(tst,net, color='orange', alpha=0.25)
        ax[num].set_title(wellname)
    vsh_net_track(selected_samples[0], logs8, var8, cutoff8, 0)
    vsh_net_track(selected_samples[1], logs8, var8, cutoff8, 1)
    vsh_net_track(selected_samples[2], logs8, var8, cutoff8, 2)
    vsh_net_track(selected_samples[3], logs8, var8, cutoff8, 3)
vsh_net_well_display(logs8, var8, cutoff8)
vsh_net_well_display(logs10, var10, cutoff10)

In [21]:
def well_display_logs(dataset, wellname, fmname, var1, var2, ref_depth, depth_step, print_flag, path):
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    data_net_khtst = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname) & 
                             ((dataset.FORMATION != 'Balakhany VIII') & (dataset.FORMATION != 'Balakhany X'))]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT']
    net_init = data[var1]*100
    net_init_calc = data_net_khtst[var1]
    net = data[var2]*100
    net_calc = data_net_khtst[var2]
    perm = data['LPERM']
    perm1 = data['PERM_DS']
    perm2 = data['PERM_US']
    khtst = data['KHtst']

    var1_sum = (net_init_calc.sum()*0.1).round(0)
    var2_sum = (net_calc.sum()*0.1).round(0)
    tst_interval = round(depth.max() - depth.min(),0)
    height = int(tst_interval/15)
    khtst_top = int(data_net_khtst['KHtst'].iloc[0])

    fig, ax = plt.subplots(1,4, figsize=(7,height), sharey=True)
    well_bal_tops = dataset[(dataset.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # ax[0].set_title((var1 + ':' + str(var1_sum) + 'm'), fontsize=10)
    ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.33, zorder=5)
    twin0.vlines(0.5, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    twin0.xaxis.set_ticks(np.arange(0, 1.5, 0.5))
    twin0.set_xlim(0, 1.5)
    twin0.grid(True)
    twin0.plot(net_init, depth, color='orange', linewidth=0.5, alpha=1)
    twin0.fill_betweenx(depth,net_init, color='orange', alpha=0.25)
    ax[1].plot(rhob, depth, color='red', alpha=0.75) 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].grid(True)
    ax[1].set_xlim(1.65, 2.65)
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 8, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue', alpha=0.75)
    twin1.set_xlim(0.6, 0)
    twin1.grid(False)
    ax[2].plot(phit, depth, color='green', linestyle='dashed')
    ax[2].set_xlim(0.3, 0)
    ax[2].xaxis.set_ticks(np.arange(0, 0.3, 0.1))
    ax[2].tick_params(labelsize=10)
    ax[2].grid(True)
    ax[2].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[2].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed', linewidth=1)
    # ax[2].set_title((var2 + ':' + str(var2_sum) + 'm'), fontsize=10)
    ax[2].set_title(var2 + ':' + str(var2_sum), fontsize=10)
    # ax[0].set_title(var1 + ':' + str(var1_sum), fontsize=10)
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5, alpha=1)
    twin2.fill_betweenx(depth, net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    twin2.xaxis.set_ticks([])
    twin2.grid(False)
    twin3 = ax[2].twiny()
    twin3.plot(rdeep, depth, color='black', linewidth=1, alpha=1)
    twin3.set_xscale('log')
    twin3.set_xlim(1, 200)
    twin3.grid(False)
    ax[3].plot(perm, depth, color='purple', alpha=0.75)
    ax[3].plot(perm1, depth, color='purple', alpha=0.5)
    ax[3].plot(perm2, depth, color='purple', alpha=0.5)
    twin4 = ax[3].twiny()
    twin4.plot(khtst, depth, color='black', alpha=1)
    ax[3].set_xscale('log')
    ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    # fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.suptitle(wellname + ' ' + fmname + ' ' 
                + ref_depth +':' + str(tst_interval) + 
                ' KHtst:' + str(khtst_top), fontsize=12)
    fig.tight_layout()
    if print_flag == 'print':
        fig.savefig(path + wellname + '.png', dpi=300)
    else:
        pass
    plt.show();
# well_display_logs(logs, 'A12X', 'Balakhany X', 'NET_VSH', 'NET', 'TST', 10, 'dontprint', 'C:\\jupyter\\SPP\\plots\\logs_vsh_bal8\\')

# PLT uploading

In [5]:
qp_oil = pd.read_csv(r'C:\jupyter\SPP_uncertainty\raw\qp_oil.csv').drop(['Id','Region'], axis=1)
qp_oil.columns = qp_oil.columns.str.lower()
qp_oil = qp_oil.rename(columns={'value':'qp_oil'})

qp_water = pd.read_csv(r'C:\jupyter\SPP_uncertainty\raw\qp_water.csv').drop(['Id','Region'], axis=1)
qp_water.columns = qp_water.columns.str.lower()
qp_water = qp_water.rename(columns={'value':'qp_water'})
qp_total = qp_water.set_index(['well','dataset','depth']).join(qp_oil.set_index(['well','dataset','depth']), how='outer').reset_index()
qp_total = qp_total.astype({'well':'string', 'dataset':'string'})
qp_total['qp_total'] = qp_total['qp_oil'] + qp_total['qp_water']
qp_total = qp_total.rename(columns={'depth':'MD'})
qp_total['MD'] = qp_total['MD'].round(1)

si_list = []
hr_list = []
for i in qp_total.dataset.unique():
    if 'SI' in i:
        si_list.append(i)
    if 'HR' in i:
        hr_list.append(i)   
exclude_list = si_list + hr_list
qp_total_v2 = qp_total[~qp_total.dataset.isin(exclude_list)]

In [ ]:
logs[logs.FORMATION_up == 'Balakhany VIII'].well.unique().shape

In [6]:
logs_wells_net = logs[['well','MD','TST','NET','FORMATION_up']]
qp_total_v2_logs = qp_total_v2.set_index(['well','MD']).join(logs_wells_net.set_index(['well','MD']), how='inner').reset_index()
qp_logs_total8 = qp_total_v2_logs[qp_total_v2_logs.FORMATION_up == 'Balakhany VIII']
qp_logs_total10 = qp_total_v2_logs[qp_total_v2_logs.FORMATION_up == 'Balakhany X']

In [ ]:
qp_logs_total10.well.unique()

In [10]:
def qp_logs_optimization8():
    qp_logs_total8_v2 = qp_logs_total8[~qp_logs_total8.well.isin(['C16','C21Z'])]
    qp_logs_total8_c16 = qp_logs_total8[(qp_logs_total8.well == 'C16') & (qp_logs_total8.dataset == 'PPROF_Mar2019_FR_6000bpd')]
    qp_logs_total8_c21z = qp_logs_total8[(qp_logs_total8.well == 'C21Z') & (qp_logs_total8.dataset == 'PPROF_March24_2400mbd')]
    qp_logs_total8_v3 = pd.concat([qp_logs_total8_v2, qp_logs_total8_c16, qp_logs_total8_c21z], axis=0).reset_index(drop=True).drop_duplicates()
    return qp_logs_total8_v3
qp_logs_total8_v3 = qp_logs_optimization8()

def qp_logs_optimization10():
    qp_logs_total10_v2 = qp_logs_total10[~qp_logs_total10.well.isin(['D05Y'])]
    qp_logs_total10_d05y = qp_logs_total10[(qp_logs_total10.well == 'D05Y') & (qp_logs_total10.dataset == 'PPROF_Oct2022_FR_2300bpd')]
    qp_logs_total10_v3 = pd.concat([qp_logs_total10_v2, qp_logs_total10_d05y], axis=0).reset_index(drop=True).drop_duplicates()
    return qp_logs_total10_v3
qp_logs_total10_v3 = qp_logs_optimization10()

In [ ]:
def vsh_net_well_display(qp_total):
    def vsh_net_track(wellname, dataset):
        fig, ax = plt.subplots(figsize=(3, 7))
        wells = dataset[(dataset.well == wellname)]
        qp_oil = wells['qp_oil']
        qp_water = wells['qp_water']
        qp = wells['qp_total']+25
        depth = wells['TST']
        net = wells['NET']*100
        fm = wells.FORMATION_up.iloc[0]
        ax.plot(qp, depth, color='red', lw=3)
        ax.plot(qp_oil, depth, color='green', lw=2)
        ax.plot(qp_water, depth, color='blue', lw=2)
        ax.invert_yaxis()
        ax2 = ax.twiny()
        ax2.plot(net, depth, color='orange', lw=0.25)
        ax2.fill_betweenx(depth, net, color='orange', alpha=0.25)
        ax2.set_xlim(0, 1)
        ax.set_title(wellname + ' ' + fm)
    for wellname in qp_total.well.unique():
        vsh_net_track(wellname, qp_total)
# vsh_net_well_display(qp_logs_total8_v3[qp_logs_total8_v3.well.isin(['B27Z','B29'])])
vsh_net_well_display(qp_logs_total10_v3[qp_logs_total10_v3.well.isin(['C25Z', 'A11Z'])])

In [27]:
def pprof_groupby(dataset, blacklist):
    dataset['NET_pprof'] = 0
    dataset.loc[dataset.qp_total > 0, 'NET_pprof'] = 1
    qp_total_gb = dataset.groupby('well')[['NET','NET_pprof']].sum().reset_index()
    qp_total_gb['NET'] = qp_total_gb['NET']*0.1
    qp_total_gb['NET_pprof'] = qp_total_gb['NET_pprof']*0.1
    qp_total_gb = qp_total_gb[~qp_total_gb.well.isin(blacklist)]
    return qp_total_gb
qp_total8_gb = pprof_groupby(qp_logs_total8_v3, ['B39', 'C16'])
qp_total10_gb = pprof_groupby(qp_logs_total10_v3, [])

In [ ]:
qp_total8_gb

In [ ]:
def regress_line(dataset, var):
    data = dataset.sort_values(by='NET').reset_index(drop=True)
    data = data[data.NET_pprof > 0]
    x = data.NET
    y = data[var]   
    coefficients = np.polyfit(x, y, 1)
    polynomial = np.poly1d(coefficients)
    x_new = [i for i in range(10, 80, 30)]
    y_fit = polynomial(x)
    y_fit_new = polynomial(x_new)
    r2 = r2_score(y, y_fit).round(2)
    return x_new, y_fit_new, coefficients, r2
x_new8, y_fit8, coef8, r2_8= regress_line(qp_total8_gb, 'NET_pprof')
x_new10, y_fit10, coef10, r2_10= regress_line(qp_total10_gb, 'NET_pprof')

def display_plotly(dataset, x_new, y_fit, p10_offset, p90_offset, title, printname):
    fig = go.Figure()
    wells = dataset.well.unique()
    x = dataset['NET'].round(0)
    y = dataset['NET_pprof'].round(0)
    hover_text = ['Well: {}<br>NET: {}<br>NET_pprof: {}'.format(w, nx, ny) 
                    for w, nx, ny in zip(wells, x, y)]
    fig.add_trace(go.Scatter(   x=x, y=y, mode='markers', 
                                hovertext=hover_text, hoverinfo='text',
                                marker=dict(color= '#17d485', size=10, opacity=0.75,
                                            line=dict(color='black', width=1)),
                                name='Well'))
    fig.add_trace(go.Scatter(   x=[0,70], y=[0,70], mode='lines',
                                line=dict(color='black', dash='dash'),
                                name='1-1 line'))
    fig.add_trace(go.Scatter(   x=[0-p10_offset,70-p10_offset], y=[0,70], mode='lines',
                                line=dict(color='green', dash='dot'),
                                name='p10 corr'))
    fig.add_trace(go.Scatter(   x=[0-p90_offset,70-p90_offset], y=[0,70], mode='lines',
                                line=dict(color='red', dash='dot'),
                                name='p90 corr'))
    # fig.add_trace(go.Scatter(   x=x_new, y=y_fit, mode='lines',
    #                             line=dict(color='blue'),
    #                             name='regression'))
    fig.update_layout(  autosize=False, 
                        width=800, height=600,
                        margin=dict(l=10, r=20, t=50, b=30),
                        showlegend=True,
                        xaxis_title='NET',
                        yaxis_title='NET_pprof',
                        title=title)
    pio.write_html(fig, r'C:\\jupyter\\SPP_uncertainty\\report\\' + printname + '.html')
    fig.show()

print('bal8 regress coef', coef8, 'R2:', r2_8)
display_plotly(qp_total8_gb, x_new8, y_fit8, 2, -7, 'PPROF vs NET Bal8', 'pprof_net_bal8')

print('bal10 regress coef', coef10, 'R2', r2_10)
display_plotly(qp_total10_gb, x_new10, y_fit10, 5, -7, 'PPROF vs NET Bal10', 'pprof_net_bal10')

# Phit_cutoff_uncert

In [30]:
# dataset = logs[logs.FORMATION_up == 'Balakhany VIII']
# dataset = dataset[(dataset.phit_flag == 1) & ((dataset.NET_VSH == 1))]
# # dataset['NET_test'] = dataset['NET']
# cutoff = 0.13
# var = var = 'NET' + '_' + str(cutoff).split('.')[1]
# dataset[var] = 0
# dataset.loc[dataset.PHIT >= cutoff, var] = 1
# new_net = dataset.groupby(['well','FORMATION_up'])[var].sum().reset_index()
# net = dataset.groupby(['well','FORMATION_up'])['NET'].sum().reset_index()
# net['NET'] = net['NET']*0.1
# new_net[var] = new_net[var]*0.1
# result = net.merge(new_net, on=['well','FORMATION_up'], how='inner')
# sns.scatterplot(data=result, x='NET', y=var, s=50)
# sns.lineplot(x=[0,100], y=[0,100], color='red')
# result

In [31]:
logs8 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v3.csv').drop('Unnamed: 0', axis=1)
logs10 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_v3.csv').drop('Unnamed: 0', axis=1)
logs = pd.concat([logs8, logs10], axis=0)

In [ ]:
def net_phit(dataset, cutoff_vsh, cutoff_phit):
    var = 'NET' + '_' + str(cutoff_phit).split('.')[1]
    dataset = dataset[(dataset.phit_flag == 1) & ((dataset.VSH <= cutoff_vsh))]
    dataset[var] = 0
    dataset.loc[dataset.PHIT >= cutoff_phit, var] = 1
    new_net = dataset.groupby(['well','FORMATION_up'])[var].sum().reset_index()
    net = dataset.groupby(['well','FORMATION_up'])['NET'].sum().reset_index()
    net['NET'] = net['NET']*0.1
    new_net[var] = new_net[var]*0.1
    result = net.merge(new_net, on=['well','FORMATION_up'], how='inner')
    return result, var, dataset, cutoff_phit
bal8, var8, logs8, cutoff8 = net_phit(logs[logs.FORMATION_up == 'Balakhany VIII'], 0.55, 0.12)
bal10, var10, logs10, cutoff10 = net_phit(logs[logs.FORMATION_up == 'Balakhany X'], 0.575, 0.11)

def xplot_net_phit(bal8, var8, bal10, var10):
    fig, ax = plt.subplots(1,2, figsize=(15, 7))
    # sns.scatterplot(data=bal8[bal8.NET > 20], x=var8, y='NET', s=40, ax=ax[0], c='green', alpha=0.5, ec='black')
    sns.kdeplot(data=bal8[bal8.NET > 20], x='NET', y=var8, fill=False, ax=ax[0], color='green', alpha=0.5)
    sns.lineplot(x=[18, 80], y=[20, 82], color='red', linestyle='dashed', ax=ax[0])
    sns.lineplot(x=[20, 80], y=[20, 80], color='black', linestyle='dashed', ax=ax[0])
    sns.lineplot(x=[27, 80], y=[20, 73], color='red', linestyle='dashed', ax=ax[0])

    # sns.scatterplot(data=bal10[bal10.NET < 70], x=var10, y='NET', s=30, ax=ax[1], c='blue', alpha=0.5, ec='gray')
    sns.kdeplot(data=bal10[bal10.NET < 70], x='NET', y=var10, s=40, ax=ax[1], c='blue', alpha=0.5)
    sns.lineplot(x=[-5, 65], y=[0, 70], color='red', linestyle='dashed', ax=ax[1])
    sns.lineplot(x=[0, 65], y=[0, 65], color='black', linestyle='dashed', ax=ax[1])
    sns.lineplot(x=[7, 65], y=[0, 58], color='red', linestyle='dashed', ax=ax[1])
    ax[0].text(25, 40, 'y=x+2', fontsize=10, c='red')
    ax[0].text(33, 30, 'line 1=1')
    ax[0].text(50, 40, 'y=x-7', fontsize=10, c='red')
    ax[0].grid(True)
    ax[1].text(15, 30, 'y=x+5', fontsize=10, c='red')
    ax[1].text(23, 20, 'line 1=1')
    ax[1].text(40, 30, 'y=x-7', fontsize=10, c='red')
    ax[1].grid(True)
xplot_net_phit(bal8, var8, bal10, var10)

In [ ]:
bal8, var8, logs8, cutoff8 = net_phit(logs[logs.FORMATION_up == 'Balakhany VIII'], 0.49, 0.18)
bal10, var10, logs10, cutoff10 = net_phit(logs[logs.FORMATION_up == 'Balakhany X'], 0.45, 0.18)

def xplot_net_phit(bal8, var8, bal10, var10):
    fig, ax = plt.subplots(1,2, figsize=(15, 7))
    # sns.scatterplot(data=bal8[bal8.NET > 20], x=var8, y='NET', s=40, ax=ax[0], c='green', alpha=0.5, ec='black')
    sns.kdeplot(data=bal8[bal8.NET > 20], x='NET', y=var8, fill=False, ax=ax[0], color='green', alpha=0.5)
    sns.lineplot(x=[18, 80], y=[20, 82], color='red', linestyle='dashed', ax=ax[0])
    sns.lineplot(x=[20, 80], y=[20, 80], color='black', linestyle='dashed', ax=ax[0])
    sns.lineplot(x=[27, 80], y=[20, 73], color='red', linestyle='dashed', ax=ax[0])

    # sns.scatterplot(data=bal10[bal10.NET < 70], x=var10, y='NET', s=30, ax=ax[1], c='blue', alpha=0.5, ec='gray')
    sns.kdeplot(data=bal10[bal10.NET < 70], x='NET', y=var10, s=40, ax=ax[1], c='blue', alpha=0.5)
    sns.lineplot(x=[-5, 65], y=[0, 70], color='red', linestyle='dashed', ax=ax[1])
    sns.lineplot(x=[0, 65], y=[0, 65], color='black', linestyle='dashed', ax=ax[1])
    sns.lineplot(x=[7, 65], y=[0, 58], color='red', linestyle='dashed', ax=ax[1])
    ax[0].text(25, 40, 'y=x+2', fontsize=10, c='red')
    ax[0].text(33, 30, 'line 1=1')
    ax[0].text(50, 40, 'y=x-7', fontsize=10, c='red')
    ax[0].grid(True)
    ax[1].text(15, 30, 'y=x+5', fontsize=10, c='red')
    ax[1].text(23, 20, 'line 1=1')
    ax[1].text(40, 30, 'y=x-7', fontsize=10, c='red')
    ax[1].grid(True)
xplot_net_phit(bal8, var8, bal10, var10)

In [121]:
logs8_full = logs[logs.FORMATION_up == 'Balakhany VIII']
logs8_join = logs8_full.set_index(['well','MD']).join(logs8[['well','MD', var8]].set_index(['well','MD']), how='left').reset_index()
logs8_join = logs8_join[logs8_join.phit_flag == 1]

In [ ]:
def phit_net_well_display(logs8, var8, cutoff8):
    my_list = list(logs8.well.unique())
    selected_samples = random.sample(my_list, 4)
    fig, ax = plt.subplots(1,4, figsize=(10, 7))
    def phit_net_track(wellname, dataset, var, cutoff, num):
        wells = dataset[(dataset.well == wellname)].iloc[200:700]
        phit = wells['PHIT']
        tst = wells['TST']
        net_init = wells['NET']*0.13
        net = wells[var]*cutoff8
        ax[num].plot(phit, tst, color='green', zorder=3)
        ax[num].plot(net_init, tst, color='black', lw=0.5)
        ax[num].plot(net, tst, color='black', lw=0.5)
        ax[num].fill_betweenx(tst,net_init, color='red', alpha=0.25)
        ax[num].fill_betweenx(tst,net, color='orange', alpha=0.25)
        ax[num].set_xlim(0.3, 0)
        ax[num].set_title(wellname)
    phit_net_track(selected_samples[0], logs8, var8, cutoff8, 0)
    phit_net_track(selected_samples[1], logs8, var8, cutoff8, 1)
    phit_net_track(selected_samples[2], logs8, var8, cutoff8, 2)
    phit_net_track(selected_samples[3], logs8, var8, cutoff8, 3)
phit_net_well_display(logs8_join, var8, cutoff8)
# phit_net_well_display(logs10, var10, cutoff10)